In [ ]:
library(TwoSampleMR)
library(MRInstruments)
library(data.table)
library(tidyverse)
library(coloc)
library(readr)
source('pathway/Get_MR-main/2.0/Get_MR2.0.r')
###################
setwd('you/pathway/CJD')

In [ ]:
## input outcome
phenofile<-"GCST90001389_buildGRCh37_rsID.tsv.gz" 
pheno <- fread(phenofile) %>% select(name,chromosome,base_pair_location,effect_allele,other_allele,beta,standard_error,p_value,effect_allele_frequency)
colnames(pheno) <- c('SNP','Chr','Pos','A1_effect','A2_other','beta_EUR','se_EUR','P_EUR','EAF_EUR') 
pheno <- pheno[order(pheno$Chr,pheno$Pos),]
pheno$CHR = "chr"
pheno<-pheno %>% unite("Chrom",CHR,Chr,sep = "",remove = FALSE)



In [ ]:
## input pqtlfile
pqtlfile <- 'BannerBBDP_DLPFC_pQTLs.csv'
protein<-fread(pqtlfile)
head(protein)
protein<-subset(protein,genes == 'STX6')

In [ ]:
## if lack of rsID then add rsID
protein$`chromosome:start` <- paste(protein$CHR, protein$POS,sep=':')
hg19 <- fread(file='/media/dai/devC/.jmm_project/brain_gut_MR_project/data/snp150_hg19.txt.gz',header=T,sep='\t',data.table=F)
data_hg19=dplyr::left_join(protein,hg19,by="chromosome:start")

In [ ]:
## change the format
protein<-protein[complete.cases(protein),]
exposure <- format_data(
 data_hg19,
 type='exposure',
 snp_col = 'name',
 beta_col = "BETA",
 se_col = "SE",
 effect_allele_col ="A1",
 other_allele_col = "REF",
 pval_col = "P",
 chr_col = 'X.CHROM',
    pos_col = 'POS',
    id_col = 'chromosome:start'
  )


In [ ]:
## if lack of eaf
eaf_gwas <- get_eaf_from_1000G(exposure,'LD_data_and_frep_1000g',type='exposure')
QTL_data1 <- eaf_gwas[complete.cases(eaf_gwas),]
QTL_data1$CHR_name <- "chr"
QTL_data1 <- QTL_data1 %>% unite("Chrom",CHR_name,CHR,sep = "",remove = FALSE)


In [ ]:
################### make lead SNP data
data_cut8 <- subset(QTL_data1,QTL_data1$pval.exposure<5e-08) 
exposure_clump <- clean_expo(data_cut8, 5e-08, clump = TRUE, kb = 10000, r2 = 0.001,LD_file='LD_data_and_frep_1000g/1kg.v3/EUR')
## add the leadSNP location
exposure_clump<-read.csv(leadfile,header = T)
exposure_clump$CHR1 <- "chr"
exposure_clump<-newlead %>% unite("Chrom",CHR1,CHR,sep = "",remove = FALSE)
exposure_clump$start <- exposure_clump$pos.exposure-1000000
exposure_clump$end <- exposure_clump$pos.exposure+1000000

write.csv(exposure_clump,file='pQTL_lead_SNP_stx6.csv')


In [ ]:

### if more than 1 leadSNP
for (i in 1:length(exposure_clump$start)){
  filename <- paste0(exposure_clump$SNP[i],"-locus.csv")
  temp<-QTL_data1[which(QTL_data1$Chrom == exposure_clump$Chrom[i]),]
  temp<-temp[which(temp$pos.exposure >= exposure_clump$start[i] & temp$pos.exposure <= exposure_clump$end[i]),]
  write.csv(temp,filename,quote = FALSE,row.names = FALSE)
}



In [ ]:
## colocalization!
locifile<-list.files(pattern = "*locus.csv")
for (t in 1:length(locifile)){
  rfile  <- locifile[t]
  loci<-read.csv(rfile,header = T)
  if(dim(loci)[1]==0){
    next
  }
  input<-merge(loci,pheno,by.x = "SNP",by.y = "SNP",suffixes
               =c("_loci","_pheno"))
  input = input[order(input$SNP,input$pval.exposure),] 
  input=input %>% distinct(input$SNP, .keep_all = TRUE) 
  ## harmonise
  for (linenum in 1:length(input$SNP)){
    ifelse(input$effect_allele.exposure[linenum] == input$A1_effect[linenum], next, input$beta.exposure[linenum] <- -as.numeric(input$beta.exposure[linenum]))
  }
  wfile<-gsub("-locus.csv","-locus-coloc.csv",rfile)
  write.csv(input,wfile,quote = FALSE,row.names = FALSE) 
    
  ## coloc   ###'cc's=case/(case+controls)
  mypheno = list(pvalues=input$P_EUR, varbeta = input$se_EUR^2,MAF = input$EAF_EUR,beta = input$beta_EUR,type="cc",s=0.232479, N=17679,snp = input$SNP)
  mypqtl = list(pvalues
              =input$pval.exposure,beta = input$beta.exposure,varbeta = input$se.exposure^2,MAF = input$eaf.exposure,type = "quant",N = 380,snp = input$SNP)
  my.res<-coloc.abf(dataset1 = mypqtl,dataset2 = mypheno)
  lociname<-gsub("-locus.csv","",rfile)
  df<-as.data.frame(my.res$summary)
  colnames(df)<-lociname
  df<-t(df)
  write.table(df,"coloc-res.tsv",sep = "\t", append = TRUE,quote = FALSE,row.names = TRUE)
}


